# Gait analysis

In [ ]:
import os

from paradigma.config import IMUConfig, GaitFeatureExtractionConfig, GaitDetectionConfig, \
    ArmActivityFeatureExtractionConfig, FilteringGaitConfig, ArmSwingQuantificationConfig
from paradigma.gait.gait_analysis import extract_gait_features_io, detect_gait_io, \
    extract_arm_activity_features_io, filter_gait_io, quantify_arm_swing_io, aggregate_quantification_dict_io
from paradigma.preprocessing import preprocess_imu_data_io

In [ ]:
branch = 'gait'
sensor = 'imu'

gait_detection_classifier_filename = 'gait_detection_classifier.pkl'
gait_detection_scaler_filename = 'gait_detection_scaler_params.json'
gait_detection_threshold_filename = 'gait_detection_threshold.txt'

gait_filtering_classifier_filename = 'gait_filtering_classifier.pkl'
gait_filtering_scaler_filename = 'gait_filtering_scaler_params.json'
gait_filtering_threshold_filename = 'gait_filtering_threshold.txt'

path_to_data =  '../../../tests/data'

path_to_classifier_input = os.path.join(path_to_data, '0.classification', branch)
path_to_classifiers = os.path.join(path_to_classifier_input, 'classifiers')
path_to_scalers = os.path.join(path_to_classifier_input, 'scalers')
path_to_thresholds = os.path.join(path_to_classifier_input, 'thresholds')

path_to_sensor_data = os.path.join(path_to_data, '1.prepared_data', sensor)
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', sensor)
path_to_extracted_features = os.path.join(path_to_data, '3.extracted_features', branch)
path_to_predictions = os.path.join(path_to_data, '4.predictions', branch)
path_to_quantification = os.path.join(path_to_data, '5.quantification', branch)
path_to_aggregation = os.path.join(path_to_data, '6.aggregation', branch)

quantification_filename = 'arm_swing_quantification.json'
aggregation_filename = 'arm_swing_aggregates.json'

# Cell has the tag 'parameters', so it won't overwrite the parameters when running the notebook in tests

## Preprocessing

In [ ]:
config = IMUConfig()

preprocess_imu_data_io(
    path_to_input=path_to_sensor_data, 
    path_to_output=path_to_preprocessed_data, 
    config=config, 
    sensor='both'
)

## Extract gait features

In [ ]:
config = GaitFeatureExtractionConfig()

extract_gait_features_io(
        config=config,
        path_to_input=path_to_preprocessed_data, 
        path_to_output=path_to_extracted_features, 
    )

## Detect gait

In [ ]:
config = GaitDetectionConfig()

full_path_to_classifier = os.path.join(path_to_classifiers, gait_detection_classifier_filename)
full_path_to_scaler = os.path.join(path_to_scalers, gait_detection_scaler_filename)

detect_gait_io(
    config=config, 
    path_to_input=path_to_extracted_features, 
    path_to_output=path_to_predictions, 
    full_path_to_classifier=full_path_to_classifier, 
    full_path_to_scaler=full_path_to_scaler
)

## Extract arm actvitiy features

In [ ]:
config = ArmActivityFeatureExtractionConfig()
full_path_to_threshold = os.path.join(path_to_thresholds, gait_detection_threshold_filename)

extract_arm_activity_features_io(
        config=config,
        path_to_timestamp_input=path_to_preprocessed_data, 
        path_to_prediction_input=path_to_predictions, 
        path_to_output=path_to_extracted_features,
        full_path_to_threshold=full_path_to_threshold
    )

## Filter gait

In [ ]:
config = FilteringGaitConfig()
full_path_to_classifier = os.path.join(path_to_classifiers, gait_filtering_classifier_filename)
full_path_to_scaler = os.path.join(path_to_scalers, gait_filtering_scaler_filename)

filter_gait_io(
    config=config,
    path_to_input=path_to_extracted_features, 
    path_to_output=path_to_predictions,
    full_path_to_classifier=full_path_to_classifier, 
    full_path_to_scaler=full_path_to_scaler
)

## Quantify arm swing

In [ ]:
asq_config = ArmSwingQuantificationConfig()
imu_config = IMUConfig()

imu_config.set_sensor('gyroscope')

full_path_to_output = os.path.join(path_to_quantification, quantification_filename)

quantify_arm_swing_io(
    imu_config=imu_config,
    asq_config=asq_config,
    path_to_timestamp_input=path_to_preprocessed_data, 
    path_to_prediction_input=path_to_predictions, 
    full_path_to_output=full_path_to_output, 
    full_path_to_threshold=full_path_to_threshold, 
)

## Aggregate quantification

In [ ]:
full_path_to_input = os.path.join(path_to_quantification, quantification_filename)
full_path_to_output = os.path.join(path_to_aggregation, aggregation_filename)

aggregate_quantification_dict_io(
    full_path_to_input=full_path_to_input,
    full_path_to_output=full_path_to_output,
    aggregates=['median', '95p']
)